In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("Data_Marketing_Customer_Analysis_Round3.csv")

In [ ]:
from scipy.stats import iqr
def remove_outliers(df):
    for c in df.columns:
            pct_75 = np.percentile(df[c], 75)
            pct_25 = np.percentile(df[c], 25)
            upper_bound = pct_75 + 1.5*iqr(df[c])
            lower_bound = pct_25 - 1.5*iqr(df[c])
            condition = (df[c] < upper_bound) & (df[c] > lower_bound)
            df[c] = df[c][condition]  # Filter out the outliers
    return df

# Highly correlated values
I know there are none from doing the lab in week 2, I'll do the correlation matrix anyways...

In [ ]:
# create dataframes with numerical...
df_num = df[df.select_dtypes(include=[np.number]).columns]
# ...and categorical data only
df_cat = df[df.select_dtypes(include=[object]).columns]

# define numerical column index
num_cols = df_num.columns
# define categorical column index
cat_cols = df_cat.columns

corr = df_num[num_cols].corr()

In [ ]:
thresh = 0.9
correlated_features = set()
for i in range(len(corr.columns)):
    for j in range(i):
        if abs(corr.iloc[i, j]) > thresh:
            colname = corr.columns[i]
            correlated_features.add(colname)

# Display the correlated features
print("Correlated features:", correlated_features)

In [ ]:
df_cat = df_cat.drop('effective_to_date', axis=1)

In [ ]:
# get unique identifiers of the regions

# Create a dictionary that maps each unique region to an integer
ed_map = {"high school or below": 0, "college":1, "bachelor": 1,"master":2, "doctor":3}
ed_map
# Replace the values in the "region" column with their mapped integers
df_cat['education'] = df_cat['education'].map(ed_map)

In [ ]:
# same for coverage
covs = df_cat["coverage"].unique()

cov_map = {coverage: i for i, coverage in enumerate(covs)}

df_cat["coverage"] = df_cat["coverage"].map(cov_map)

In [ ]:
# same for vehicle size
sizes = {"small" : 0, "medsize" : 1, "large" : 2}

size_map = {size: i for i, size in enumerate(sizes)}

df_cat["vehicle_size"] = df_cat["vehicle_size"].map(size_map)

In [ ]:
cols_to_dummify = df_cat.columns.drop(["coverage","education","vehicle_size"])

df_dummies = pd.get_dummies(cols_to_dummify, sparse=1,drop_first=1)
df_dummies

In [ ]:
# create dummy variables
# for all non-ordinal columns
cols_to_dummify = df_cat.columns.drop(["coverage","education","vehicle_size"])
df_dums = pd.DataFrame()
for i, col in enumerate(cols_to_dummify):
    df_dummies = pd.get_dummies(df_cat[col], drop_first=1)
    # concatenate the original dataframe with the dummy variables
    df_dums = pd.concat([df_dums, df_dummies], axis=1)

In [ ]:
df_dums

In [ ]:
df_done = pd.concat([df_cat[["coverage","education","vehicle_size"]], df_dums], axis=1)

In [ ]:
df_done

In [ ]:
df = pd.concat([df_cat, df_num], axis=1)

# Optional

In [ ]:
df_num

In [ ]:
# import required libraries
from sklearn import linear_model
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler

scale = MinMaxScaler()

X = df_num[["customer_lifetime_value"]]

scaledX = scale.fit_transform(X)

display(X, scaledX)

In [ ]:
# plot X against scaledX to verify the rescaling
# Create a figure and axis
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

# Plot the first distribution
sns.histplot(data=X, kde=True, ax=ax[0])

# Plot the second distribution
sns.histplot(data=scaledX, kde=True, ax=ax[1])

# Adjust the plot layout
#plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
print(scaledX.std(), scaledX.mean())

In [ ]:
scale = PowerTransformer(method="box-cox")

X = df_num[["customer_lifetime_value"]]

scaledX = scale.fit_transform(X)

#print(X,scaledX)

# plot X against scaledX to verify the rescaling
# Create a figure and axis
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

# Plot the first distribution
sns.histplot(data=X, kde=True, ax=ax[0])

# Plot the second distribution
sns.histplot(data=scaledX, kde=True, ax=ax[1])

# Adjust the plot layout
#plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
print(scaledX.std(), scaledX.mean())

In [ ]:
scale = PowerTransformer(method="yeo-johnson")

X = df_num[["customer_lifetime_value"]]

scaledX = scale.fit_transform(X)

#print(X,scaledX)

# plot X against scaledX to verify the rescaling
# Create a figure and axis
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

# Plot the first distribution
sns.histplot(data=X, kde=True, ax=ax[0])

# Plot the second distribution
sns.histplot(data=scaledX, kde=True, ax=ax[1])

# Adjust the plot layout
#plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
print(scaledX.std(), scaledX.mean())

The difference between the options "box-cox" and "yeo-johnson" is that the former requires strictly positive input data while the latter takes both positive and negative values.

#